In [1]:
from fl_spn.partitioner import FederatedDataPartitioner
from trainer import FederatedEiNetTrainer
from utils import load_dataset
from fl_spn.config import SupervisedFLConfig

In [2]:
data = load_dataset(name="adult")

partitioner = FederatedDataPartitioner(
    X=data["X_train"],
    y=data["y_train"],
    feature_names=data["X_processed"].columns.tolist(),
    numeric_features=data["numeric_features"],
    categorical_features=data["categorical_features"],
)

In [3]:
print("\n" + "=" * 60)
print("🔵 Test 1: Horizontal Partitioning")
print("=" * 60)

horizontal_partition = partitioner.horizontal_partition(
    num_clients=SupervisedFLConfig.num_clients,
    random_state=SupervisedFLConfig.random_seed,
)

horizontal_trainer = FederatedEiNetTrainer(horizontal_partition)

horizontal_results = horizontal_trainer.train_federated_learning(
    data["X_processed"], epochs=SupervisedFLConfig.epochs, verbose=True
)
horizontal_eval = horizontal_trainer.evaluate_on_test(
        data["X_test"], data["y_test"], data["X_processed"].columns.tolist()
    )

INFO:fl_spn.partitioner:🔄 Performing HORIZONTAL partitioning into 3 clients...
INFO:fl_spn.partitioner:  Client 0: 12059 samples, 14 features
INFO:fl_spn.partitioner:  Client 1: 12059 samples, 14 features
INFO:fl_spn.partitioner:  Client 2: 12059 samples, 14 features
INFO:trainer:
🚀 Starting horizontal federated learning...
INFO:trainer:Training parameters: epochs=10
INFO:trainer:
📍 Training client client_0...
INFO:trainer:   Data shape: 12059 samples × 14 features
INFO:trainer:   🔗 Overlapping features: 14 feature(s)- ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
INFO:simple_einet.layers.distributions.piecewise_linear:Initializing piecewise linear distribution with data of shape torch.Size([12059, 1, 14]).



🔵 Test 1: Horizontal Partitioning


Initializing PiecewiseLinear Leaf Layer: 100%|██████████| 3/3 [00:00<00:00, 11.13it/s]
INFO:trainer:    📊 Model config: depth=2, sums=12, leaves=12
INFO:trainer:    🔧 Feature domains: 14 domain created
/Users/M279402/PycharmProjects/fl_spn/.venv/lib/python3.10/site-packages/simple_einet/layers/distributions/piecewise_linear.py:263: UserWarning: torch.searchsorted(): input value tensor is non-contiguous, this will lower the performance due to extra data copy when converting non-contiguous tensor to contiguous, please use contiguous input value tensor if possible. This message will only appear once per program. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/BucketizationUtils.h:34.)
  indices = torch.searchsorted(xp, x, right=False)
INFO:trainer:Epoch:  5, Loss: 0.6823, Train Acc: 54.40%, F1: 51.24%
INFO:trainer:Epoch: 10, Loss: 0.6543, Train Acc: 57.13%, F1: 52.64%
INFO:trainer:    ✅ Training accuracy: 57.127
INFO:trainer:    📈 Training F1 score

In [4]:
print("\n" + "="*60)
print("🟢 Test 2: Vertical Partitioning")
print("="*60)

vertical_partition = partitioner.vertical_partition(
    num_clients=SupervisedFLConfig.num_clients, 
    random_state=SupervisedFLConfig.random_seed)

vertical_trainer = FederatedEiNetTrainer(vertical_partition)

vertical_results = vertical_trainer.train_federated_learning(
    data["X_processed"], epochs=SupervisedFLConfig.epochs, verbose=True
)

# 在測試集上評估
vertical_eval = vertical_trainer.evaluate_on_test(
    data["X_test"], data["y_test"], data["X_processed"].columns.tolist()
)

INFO:fl_spn.partitioner:🔄 Performing VERTICAL partitioning into 3 clients...
INFO:fl_spn.partitioner:  Client 0: 36177 samples, 4 features
INFO:fl_spn.partitioner:  Client 1: 36177 samples, 4 features
INFO:fl_spn.partitioner:  Client 2: 36177 samples, 6 features
INFO:trainer:
🚀 Starting vertical federated learning...
INFO:trainer:Training parameters: epochs=10
INFO:trainer:
📍 Training client client_0...
INFO:trainer:   Data shape: 36177 samples × 4 features
INFO:simple_einet.layers.distributions.piecewise_linear:Initializing piecewise linear distribution with data of shape torch.Size([36177, 1, 4]).



🟢 Test 2: Vertical Partitioning


Initializing PiecewiseLinear Leaf Layer: 100%|██████████| 3/3 [00:00<00:00, 27.46it/s]
INFO:trainer:    📊 Model config: depth=1, sums=8, leaves=8
INFO:trainer:    🔧 Feature domains: 4 domain created
INFO:trainer:Epoch:  5, Loss: 0.6930, Train Acc: 63.44%, F1: 60.35%
INFO:trainer:Epoch: 10, Loss: 0.6756, Train Acc: 63.45%, F1: 60.35%
INFO:trainer:    ✅ Training accuracy: 63.446
INFO:trainer:    📈 Training F1 score: 60.345
INFO:trainer:    ⏱️ Training time: 1.120 seconds
INFO:trainer:   🎯 Client client_0 training complete
INFO:trainer:
📍 Training client client_1...
INFO:trainer:   Data shape: 36177 samples × 4 features
INFO:simple_einet.layers.distributions.piecewise_linear:Initializing piecewise linear distribution with data of shape torch.Size([36177, 1, 4]).
Initializing PiecewiseLinear Leaf Layer: 100%|██████████| 3/3 [00:00<00:00, 17.51it/s]
INFO:trainer:    📊 Model config: depth=1, sums=8, leaves=8
INFO:trainer:    🔧 Feature domains: 4 domain created
INFO:trainer:Epoch:  5, Loss: 0

In [5]:
print("\n" + "=" * 60)
print("🟡 Test 3: Hybrid Partitioning")
print("=" * 60)

hybrid_partition = partitioner.hybrid_partition(
    num_clients=SupervisedFLConfig.num_clients,
    sample_overlap_ratio=SupervisedFLConfig.sample_overlap_ratio,
    feature_overlap_ratio=SupervisedFLConfig.feature_overlap_ratio,
    random_state=SupervisedFLConfig.random_seed,
)
hybrid_trainer = FederatedEiNetTrainer(hybrid_partition)
hybrid_results = hybrid_trainer.train_federated_learning(
    data["X_processed"], epochs=SupervisedFLConfig.epochs, verbose=True
)
hybrid_eval = hybrid_trainer.evaluate_on_test(
        data["X_test"], data["y_test"], data["X_processed"].columns.tolist()
    )

INFO:fl_spn.partitioner:🔄 Performing HYBRID partitioning into 3 clients...
INFO:fl_spn.partitioner:  Sample overlap ratio: 0.0%
INFO:fl_spn.partitioner:  Feature overlap ratio: 0.0%
INFO:fl_spn.partitioner:  Base samples: 18087; Overlap sample pool: 0
INFO:fl_spn.partitioner:  Base features: 6; Overlap feature pool: 0
INFO:fl_spn.partitioner:  Client 0: 6029 samples × 2 features
INFO:fl_spn.partitioner:  Client 1: 6029 samples × 2 features
INFO:fl_spn.partitioner:  Client 2: 6029 samples × 2 features
INFO:trainer:
🚀 Starting hybrid_robust federated learning...
INFO:trainer:Training parameters: epochs=10
INFO:trainer:
📍 Training client client_0...
INFO:trainer:   Data shape: 6029 samples × 2 features
INFO:simple_einet.layers.distributions.piecewise_linear:Initializing piecewise linear distribution with data of shape torch.Size([6029, 1, 2]).



🟡 Test 3: Hybrid Partitioning


Initializing PiecewiseLinear Leaf Layer: 100%|██████████| 3/3 [00:00<00:00, 73.47it/s]
INFO:trainer:    📊 Model config: depth=1, sums=4, leaves=4
INFO:trainer:    🔧 Feature domains: 2 domain created
INFO:trainer:Epoch:  5, Loss: 0.9271, Train Acc: 39.39%, F1: 37.51%
INFO:trainer:Epoch: 10, Loss: 0.8908, Train Acc: 39.56%, F1: 37.65%
INFO:trainer:    ✅ Training accuracy: 39.559
INFO:trainer:    📈 Training F1 score: 37.650
INFO:trainer:    ⏱️ Training time: 0.144 seconds
INFO:trainer:   🎯 Client client_0 training complete
INFO:trainer:
📍 Training client client_1...
INFO:trainer:   Data shape: 6029 samples × 2 features
INFO:simple_einet.layers.distributions.piecewise_linear:Initializing piecewise linear distribution with data of shape torch.Size([6029, 1, 2]).
Initializing PiecewiseLinear Leaf Layer: 100%|██████████| 3/3 [00:00<00:00, 35.23it/s]
INFO:trainer:    📊 Model config: depth=1, sums=4, leaves=4
INFO:trainer:    🔧 Feature domains: 2 domain created
INFO:trainer:Epoch:  5, Loss: 0.7

In [15]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

methods = ['Horizontal', 'Vertical', 'Hybrid']

train_acc = [horizontal_results['weighted_accuracy'], 
             vertical_results['weighted_accuracy'], 
             hybrid_results['weighted_accuracy']]

f1_acc = [horizontal_results['weighted_f1'], 
          vertical_results['weighted_f1'], 
          hybrid_results['weighted_f1']]

train_time = [horizontal_results['total_training_time'], 
              vertical_results['total_training_time'], 
              hybrid_results['total_training_time']]

clients = [horizontal_results['num_clients'], 
           vertical_results['num_clients'], 
           hybrid_results['num_clients']]

samples = [horizontal_results['total_samples'], 
           vertical_results['total_samples'], 
           hybrid_results['total_samples']]

colors = ['#F3AA60', '#EF6262', '#468897']


In [29]:
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Accuracy', 'F1',)
)

for i, method in enumerate(methods):
    # Accuracy bar
    fig.add_trace(
        go.Bar(name=f'Train {method}', x=[method], y=[train_acc[i]],
               marker_color=colors[i], opacity=0.8, legendgroup=f'group{i}',
               showlegend=False),
        row=1, col=1
    )
    # Add annotation for accuracy
    fig.add_annotation(
        x=method, y=train_acc[i], text=f"{train_acc[i]:.2f}",
        showarrow=False, font=dict(size=12, color="black"),
        xanchor='center', yanchor='bottom', row=1, col=1
    )

    # F1 bar (replace with train_f1[i] if needed)
    fig.add_trace(
        go.Bar(name=f'Train {method}', x=[method], y=[f1_acc[i]],
               marker_color=colors[i], opacity=0.8, legendgroup=f'group{i}',
               showlegend=False),
        row=1, col=2
    )
    # Add annotation for F1
    fig.add_annotation(
        x=method, y=f1_acc[i], text=f"{f1_acc[i]:.2f}",
        showarrow=False, font=dict(size=12, color="black"),
        xanchor='center', yanchor='bottom', row=1, col=2
    )

fig.update_yaxes(range=[0, 100])
fig.update_layout(
    title=f'FL Methods Performance Comparison (epochs={SupervisedFLConfig.epochs})',
)
fig.update_layout(template='plotly_white')
fig.update_yaxes(ticksuffix="%", range=[0, 100])
fig.show()

In [34]:
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Training Time', 'Client Count', 'Total Samples')
)

fig.add_trace(
    go.Bar(x=methods, y=train_time, marker_color=colors,
           name='Train Time', showlegend=False),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=methods, y=clients, marker_color=colors,
           name='Clients', showlegend=False),
    row=1, col=2
)
fig.add_trace(
    go.Bar(x=methods, y=samples, marker_color=colors,
           name='Samples', showlegend=False),
    row=1, col=3
)
fig.update_layout(
    title=f'Other Config Comparison',
)
fig.update_layout(template='plotly_white')
fig.show()